In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv(r'DOH COVID Data Drop_ 20201031 - 04 Case Information.csv')
data

,CaseCode,Age,AgeGroup,Sex,DateSpecimen,DateResultRelease,DateRepConf,DateDied,DateRecover,RemovalType,...,ProvRes,CityMunRes,CityMuniPSGC,BarangayRes,BarangayPSGC,HealthStatus,Quarantined,DateOnset,Pregnanttab,ValidationStatus
0,C642447,25.0,25 to 29,MALE,2020-07-14,2020-07-15,2020-07-19,NaN,NaN,RECOVERED,...,COTABATO CITY (NOT A PROVINCE),NaN,NaN,NaN,NaN,RECOVERED,NO,NaN,NaN,"Removal Type is ""Recovered"", but no Recovered ..."
1,C919481,54.0,50 to 54,MALE,2020-07-27,2020-07-30,2020-08-03,NaN,NaN,RECOVERED,...,NaN,NaN,NaN,NaN,NaN,RECOVERED,NO,NaN,NaN,"Health Status is ""Recovered"", but no Date Reco..."
2,C378726,50.0,50 to 54,FEMALE,2020-07-18,2020-07-19,2020-07-24,NaN,NaN,RECOVERED,...,NaN,NaN,NaN,NaN,NaN,RECOVERED,NO,NaN,NO,"Health Status is ""Recovered"", but no Date Reco..."
3,C556553,30.0,30 to 34,FEMALE,2020-05-27,2020-05-27,2020-05-31,NaN,NaN,RECOVERED,...,NaN,NaN,NaN,NaN,NaN,RECOVERED,NO,NaN,NO,"Removal Type is ""Recovered"", but no Recovered ..."
4,C210854,33.0,30 to 34,MALE,2020-07-21,2020-07-24,2020-07-28,NaN,NaN,RECOVERED,...,NaN,NaN,NaN,NaN,NaN,RECOVERED,NO,NaN,NaN,"Health Status is ""Recovered"", but no Date Reco..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380724,C881229,29.0,25 to 29,MALE,2020-09-09,2020-09-13,2020-09-16,NaN,2020-09-24,RECOVERED,...,ROMBLON,CALATRAVA,PH175904000,BALOGO,PH175904001,RECOVERED,YES,NaN,NaN,NaN
380725,C591169,22.0,20 to 24,MALE,2020-09-09,2020-09-13,2020-09-16,NaN,2020-09-24,RECOVERED,...,ROMBLON,CALATRAVA,PH175904000,BALOGO,PH175904001,RECOVERED,YES,NaN,NaN,NaN
380726,C325398,27.0,25 to 29,FEMALE,2020-09-09,2020-09-13,2020-09-16,NaN,2020-09-24,RECOVERED,...,ROMBLON,CALATRAVA,PH175904000,BALOGO,PH175904001,RECOVERED,YES,NaN,NO,Age or Birthdate is Invalid
380727,C768847,46.0,45 to 49,FEMALE,2020-08-24,2020-08-28,2020-08-30,NaN,2020-09-07,RECOVERED,...,ROMBLON,SANTA MARIA (IMELDA),PH175917000,NaN,NaN,RECOVERED,YES,2020-08-20,NO,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380729 entries, 0 to 380728
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CaseCode           380729 non-null  object 
 1   Age                378546 non-null  float64
 2   AgeGroup           378546 non-null  object 
 3   Sex                380729 non-null  object 
 4   DateSpecimen       318641 non-null  object 
 5   DateResultRelease  319409 non-null  object 
 6   DateRepConf        380729 non-null  object 
 7   DateDied           7155 non-null    object 
 8   DateRecover        91993 non-null   object 
 9   RemovalType        338267 non-null  object 
 10  Admitted           380579 non-null  object 
 11  RegionRes          377056 non-null  object 
 12  ProvRes            355798 non-null  object 
 13  CityMunRes         345275 non-null  object 
 14  CityMuniPSGC       345275 non-null  object 
 15  BarangayRes        151373 non-null  object 
 16  Ba

In [4]:
df = data.drop('CaseCode', axis=1)
df = df.drop(df.loc[:,'AgeGroup':'Sex'].columns, axis=1)
df = df.drop(df.loc[:, 'DateResultRelease':'Admitted'].columns, axis=1)
df = df.drop('ProvRes', axis=1)
df = df.drop(df.loc[:, 'CityMuniPSGC':'BarangayPSGC'].columns, axis=1)
df = df.drop(df.loc[:, 'DateOnset':'ValidationStatus'].columns, axis=1)
df

,Age,DateSpecimen,RegionRes,CityMunRes,HealthStatus,Quarantined
0,25.0,2020-07-14,Region XII: SOCCSKSARGEN,NaN,RECOVERED,NO
1,54.0,2020-07-27,NaN,NaN,RECOVERED,NO
2,50.0,2020-07-18,ROF,NaN,RECOVERED,NO
3,30.0,2020-05-27,NCR,NaN,RECOVERED,NO
4,33.0,2020-07-21,NCR,NaN,RECOVERED,NO
...,...,...,...,...,...,...
380724,29.0,2020-09-09,Region IV-B: MIMAROPA,CALATRAVA,RECOVERED,YES
380725,22.0,2020-09-09,Region IV-B: MIMAROPA,CALATRAVA,RECOVERED,YES
380726,27.0,2020-09-09,Region IV-B: MIMAROPA,CALATRAVA,RECOVERED,YES
380727,46.0,2020-08-24,Region IV-B: MIMAROPA,SANTA MARIA (IMELDA),RECOVERED,YES


In [5]:
titles = list(df.columns)
titles

['Age',
 'DateSpecimen',
 'RegionRes',
 'CityMunRes',
 'HealthStatus',
 'Quarantined']

In [6]:
titles[4], titles[5] = titles[5], titles[4]
titles[2], titles[3] = titles[3], titles[2]
titles

['Age',
 'DateSpecimen',
 'CityMunRes',
 'RegionRes',
 'Quarantined',
 'HealthStatus']

In [7]:
df = df[titles]
df

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,25.0,2020-07-14,NaN,Region XII: SOCCSKSARGEN,NO,RECOVERED
1,54.0,2020-07-27,NaN,NaN,NO,RECOVERED
2,50.0,2020-07-18,NaN,ROF,NO,RECOVERED
3,30.0,2020-05-27,NaN,NCR,NO,RECOVERED
4,33.0,2020-07-21,NaN,NCR,NO,RECOVERED
...,...,...,...,...,...,...
380724,29.0,2020-09-09,CALATRAVA,Region IV-B: MIMAROPA,YES,RECOVERED
380725,22.0,2020-09-09,CALATRAVA,Region IV-B: MIMAROPA,YES,RECOVERED
380726,27.0,2020-09-09,CALATRAVA,Region IV-B: MIMAROPA,YES,RECOVERED
380727,46.0,2020-08-24,SANTA MARIA (IMELDA),Region IV-B: MIMAROPA,YES,RECOVERED


In [8]:
start_date = '2020-09-10'
end_date = '2020-10-10'

df = df.loc[df['RegionRes'] == 'NCR']
df = df.loc[(df['DateSpecimen'] >= start_date) & ((df['DateSpecimen'] <= end_date))]
df

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
376,59.0,2020-09-10,CITY OF NAVOTAS,NCR,YES,RECOVERED
736,32.0,2020-09-28,CITY OF NAVOTAS,NCR,YES,RECOVERED
856,39.0,2020-10-02,CITY OF NAVOTAS,NCR,YES,RECOVERED
1167,46.0,2020-09-10,CITY OF NAVOTAS,NCR,YES,RECOVERED
1184,59.0,2020-09-22,CITY OF MANILA,NCR,NO,RECOVERED
...,...,...,...,...,...,...
375985,24.0,2020-09-25,TAGUIG CITY,NCR,NO,RECOVERED
375989,48.0,2020-09-24,TAGUIG CITY,NCR,NO,RECOVERED
376033,41.0,2020-09-23,TAGUIG CITY,NCR,NO,RECOVERED
376034,57.0,2020-09-28,TAGUIG CITY,NCR,NO,RECOVERED


In [9]:
#Cities in NCR
cities = df['CityMunRes'].unique().tolist()

cities

['CITY OF NAVOTAS',
 'CITY OF MANILA',
 nan,
 'CITY OF MALABON',
 'CITY OF PASIG',
 'CALOOCAN CITY',
 'QUEZON CITY',
 'CITY OF SAN JUAN',
 'CITY OF PARAÑAQUE',
 'CITY OF MAKATI',
 'CITY OF MARIKINA',
 'CITY OF MANDALUYONG',
 'CITY OF VALENZUELA',
 'CITY OF LAS PIÑAS',
 'CITY OF MUNTINLUPA',
 'PASAY CITY',
 'PATEROS',
 'TAGUIG CITY']

In [10]:
hpStats = df['HealthStatus'].unique().tolist()
hpStats

['RECOVERED', 'MILD', 'SEVERE', 'DIED', 'ASYMPTOMATIC', 'CRITICAL']

In [11]:
#df = df.loc[(df['CityMunRes'] == 'QUEZON CITY') | (df['CityMunRes'] == 'CITY OF MAKATI')]
df = df.reset_index(drop=True)
df['HealthStatus'].replace(to_replace=['MILD','SEVERE','ASYMPTOMATIC','CRITICAL'],value='SICK',inplace=True)
hpStats = df['HealthStatus'].unique().tolist()

hpStats

['RECOVERED', 'SICK', 'DIED']

In [12]:
df = df.dropna()
df = df.reset_index(drop=True)
df

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,59.0,2020-09-10,CITY OF NAVOTAS,NCR,YES,RECOVERED
1,32.0,2020-09-28,CITY OF NAVOTAS,NCR,YES,RECOVERED
2,39.0,2020-10-02,CITY OF NAVOTAS,NCR,YES,RECOVERED
3,46.0,2020-09-10,CITY OF NAVOTAS,NCR,YES,RECOVERED
4,59.0,2020-09-22,CITY OF MANILA,NCR,NO,RECOVERED
...,...,...,...,...,...,...
20606,24.0,2020-09-25,TAGUIG CITY,NCR,NO,RECOVERED
20607,48.0,2020-09-24,TAGUIG CITY,NCR,NO,RECOVERED
20608,41.0,2020-09-23,TAGUIG CITY,NCR,NO,RECOVERED
20609,57.0,2020-09-28,TAGUIG CITY,NCR,NO,RECOVERED


In [13]:
df['DateSpecimen'] = pd.to_datetime(df['DateSpecimen'], format='%Y-%m-%d')
df

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,59.0,2020-09-10,CITY OF NAVOTAS,NCR,YES,RECOVERED
1,32.0,2020-09-28,CITY OF NAVOTAS,NCR,YES,RECOVERED
2,39.0,2020-10-02,CITY OF NAVOTAS,NCR,YES,RECOVERED
3,46.0,2020-09-10,CITY OF NAVOTAS,NCR,YES,RECOVERED
4,59.0,2020-09-22,CITY OF MANILA,NCR,NO,RECOVERED
...,...,...,...,...,...,...
20606,24.0,2020-09-25,TAGUIG CITY,NCR,NO,RECOVERED
20607,48.0,2020-09-24,TAGUIG CITY,NCR,NO,RECOVERED
20608,41.0,2020-09-23,TAGUIG CITY,NCR,NO,RECOVERED
20609,57.0,2020-09-28,TAGUIG CITY,NCR,NO,RECOVERED


In [14]:
df['DateSpecimen'] = df['DateSpecimen'].dt.strftime('%m/%d/%Y')
df

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,59.0,09/10/2020,CITY OF NAVOTAS,NCR,YES,RECOVERED
1,32.0,09/28/2020,CITY OF NAVOTAS,NCR,YES,RECOVERED
2,39.0,10/02/2020,CITY OF NAVOTAS,NCR,YES,RECOVERED
3,46.0,09/10/2020,CITY OF NAVOTAS,NCR,YES,RECOVERED
4,59.0,09/22/2020,CITY OF MANILA,NCR,NO,RECOVERED
...,...,...,...,...,...,...
20606,24.0,09/25/2020,TAGUIG CITY,NCR,NO,RECOVERED
20607,48.0,09/24/2020,TAGUIG CITY,NCR,NO,RECOVERED
20608,41.0,09/23/2020,TAGUIG CITY,NCR,NO,RECOVERED
20609,57.0,09/28/2020,TAGUIG CITY,NCR,NO,RECOVERED


In [15]:
qc = df.loc[df['CityMunRes'] == 'CITY OF MALABON']
mkt = df.loc[df['CityMunRes'] == 'CITY OF MAKATI']

qc = qc.reset_index(drop=True)
mkt = mkt.reset_index(drop=True)

In [16]:
qc

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,29.0,09/18/2020,CITY OF MALABON,NCR,NO,RECOVERED
1,51.0,09/30/2020,CITY OF MALABON,NCR,NO,RECOVERED
2,22.0,09/18/2020,CITY OF MALABON,NCR,NO,RECOVERED
3,29.0,09/30/2020,CITY OF MALABON,NCR,NO,RECOVERED
4,63.0,09/13/2020,CITY OF MALABON,NCR,NO,RECOVERED
...,...,...,...,...,...,...
446,33.0,09/23/2020,CITY OF MALABON,NCR,NO,RECOVERED
447,32.0,09/23/2020,CITY OF MALABON,NCR,NO,RECOVERED
448,19.0,09/16/2020,CITY OF MALABON,NCR,NO,RECOVERED
449,76.0,09/30/2020,CITY OF MALABON,NCR,NO,DIED


In [17]:
mkt

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,44.0,09/14/2020,CITY OF MAKATI,NCR,NO,RECOVERED
1,35.0,09/23/2020,CITY OF MAKATI,NCR,NO,RECOVERED
2,25.0,09/22/2020,CITY OF MAKATI,NCR,NO,RECOVERED
3,59.0,09/11/2020,CITY OF MAKATI,NCR,NO,RECOVERED
4,51.0,09/24/2020,CITY OF MAKATI,NCR,YES,RECOVERED
...,...,...,...,...,...,...
1457,28.0,09/13/2020,CITY OF MAKATI,NCR,NO,RECOVERED
1458,24.0,09/10/2020,CITY OF MAKATI,NCR,NO,RECOVERED
1459,59.0,09/16/2020,CITY OF MAKATI,NCR,NO,RECOVERED
1460,30.0,10/05/2020,CITY OF MAKATI,NCR,NO,RECOVERED


In [18]:
#1st City = QC
qcHealth = qc['HealthStatus'].unique().tolist()
qcDate = qc['DateSpecimen'].unique().tolist()
qcQuar = qc['Quarantined'].unique().tolist()
qcCity = qc['CityMunRes'].unique().tolist()
qcReg = qc['RegionRes'].unique().tolist()

#2nd City = MKT
mktHealth = mkt['HealthStatus'].unique().tolist()
mktDate = mkt['DateSpecimen'].unique().tolist()
mktQuar = mkt['Quarantined'].unique().tolist()
mktCity = mkt['CityMunRes'].unique().tolist()
mktReg = mkt['RegionRes'].unique().tolist()

#Overall
uniHealth = df['HealthStatus'].unique().tolist()
uniDate = df['DateSpecimen'].unique().tolist()
uniQuar = df['Quarantined'].unique().tolist()
uniCity = df['CityMunRes'].unique().tolist()

In [19]:
#1st City = QC
for i in range(len(qc)):
    for l in qcHealth:
        if qc['HealthStatus'][i] == l:
            qc['HealthStatus'][i] = qcHealth.index(l)
            
    for m in qcDate:
        if qc['DateSpecimen'][i] == m:
            qc['DateSpecimen'][i] = qcDate.index(m)
    
    for n in qcQuar:
        if qc['Quarantined'][i] == n:
            qc['Quarantined'][i] = qcQuar.index(n)
        
    for o in qcCity:
        if qc['CityMunRes'][i] == o:
            qc['CityMunRes'][i] = qcCity.index(o)
    
    for p in qcReg:
        if qc['RegionRes'][i] == p:
            qc['RegionRes'][i] = qcReg.index(p)
   
#2nd City = Mkt
for i in range(len(mkt)):
    for l in mktHealth:
        if mkt['HealthStatus'][i] == l:
            mkt['HealthStatus'][i] = mktHealth.index(l)
            
    for m in uniDate:
        if mkt['DateSpecimen'][i] == m:
            mkt['DateSpecimen'][i] = mktDate.index(m)
    
    for n in mktQuar:
        if mkt['Quarantined'][i] == n:
            mkt['Quarantined'][i] = mktQuar.index(n)

    for o in mktCity:
        if mkt['CityMunRes'][i] == o:
            mkt['CityMunRes'][i] = mktCity.index(o)
    
    for p in mktReg:
        if mkt['RegionRes'][i] == p:
            mkt['RegionRes'][i] = mktReg.index(p)

In [20]:
#Overall
for i in range(len(df)):
    for l in uniHealth:
        if df['HealthStatus'][i] == l:
            df['HealthStatus'][i] = uniHealth.index(l)
            
    for m in uniDate:
        if df['DateSpecimen'][i] == m:
            df['DateSpecimen'][i] = uniDate.index(m)

    for n in uniQuar:
        if df['Quarantined'][i] == n:
            df['Quarantined'][i] = uniQuar.index(n)
    
    for o in uniCity:
        if df['CityMunRes'][i] == o:
            df['CityMunRes'][i] = uniCity.index(o)

In [21]:
qc

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
0,29.0,0,0,0,0,0
1,51.0,1,0,0,0,0
2,22.0,0,0,0,0,0
3,29.0,1,0,0,0,0
4,63.0,2,0,0,0,0
...,...,...,...,...,...,...
446,33.0,9,0,0,0,0
447,32.0,9,0,0,0,0
448,19.0,13,0,0,0,0
449,76.0,1,0,0,0,1


In [22]:
qc[['HealthStatus', 'DateSpecimen', 'Quarantined', 'CityMunRes', 'RegionRes']] = qc[['HealthStatus', 'DateSpecimen', 'Quarantined', 'CityMunRes', 'RegionRes']].apply(pd.to_numeric)
mkt[['HealthStatus', 'DateSpecimen', 'Quarantined', 'CityMunRes', 'RegionRes']] = mkt[['HealthStatus', 'DateSpecimen', 'Quarantined', 'CityMunRes', 'RegionRes']].apply(pd.to_numeric)

#combined
df[['HealthStatus', 'DateSpecimen', 'Quarantined', 'CityMunRes']] = df[['HealthStatus', 'DateSpecimen', 'Quarantined', 'CityMunRes']].apply(pd.to_numeric)

In [23]:
qc.corr(method='pearson')

,Age,DateSpecimen,CityMunRes,RegionRes,Quarantined,HealthStatus
Age,1.000000,0.126120,NaN,NaN,-0.052712,0.056335
DateSpecimen,0.126120,1.000000,NaN,NaN,0.015937,0.465905
CityMunRes,NaN,NaN,NaN,NaN,NaN,NaN
RegionRes,NaN,NaN,NaN,NaN,NaN,NaN
Quarantined,-0.052712,0.015937,NaN,NaN,1.000000,0.111829
HealthStatus,0.056335,0.465905,NaN,NaN,0.111829,1.000000


In [24]:
cols = qc.loc[:, 'Age':'Quarantined'].columns.to_list()
x = qc[['Age','Quarantined']]
y = qc['HealthStatus']

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [26]:
clf = DecisionTreeClassifier()

In [27]:
#logreg.fit(x_train, y_train)
#logreg2.fit(x_train2, y_train2)
clf = clf.fit(x_train,y_train)

In [28]:
y_pred = clf.predict([[94,1]]) #if age = 84, dead
y_pred

array([2], dtype=int64)

In [29]:
def prediction(x,y):
    tree = DecisionTreeClassifier()
    tree = tree.fit(x_test,y_test)
    y_pred = clf.predict([[x,y]])
    return y_pred

In [30]:
userInput = input('Enter Quarantine Status: ')
userInput = userInput.lower()
toPred = -1

if userInput == 'yes':
    toPred = 0
    
elif userInput == 'no':
    toPred = 1

prediction(21,toPred)

array([0], dtype=int64)

In [31]:
matrix = metrics.confusion_matrix(y_test, y_pred)
matrix

ValueError: Found input variables with inconsistent numbers of samples: [91, 1]

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.heatmap(pd.DataFrame(matrix), annot=True, cmap="coolwarm",fmt='g')
ax.xaxis.set_label_position("top")
ax.set_xticklabels([qcHealth[0],mktHealth[1],mktHealth[2]])
ax.set_yticklabels([qcHealth[0],mktHealth[1],mktHealth[2]])
plt.title('Matrix')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sns.regplot(qc['HealthStatus'],qc['Age'],data=qc)